In [ ]:
import pandas as pd
import numpy as np
import os

# Define the path
BASE_PATH = r"C:\Users\NyashaKampila\Desktop\Projects\Purchase-Order-Creation-Project"
INPUT_FILE = os.path.join(BASE_PATH, "PO Data.csv")
OUTPUT_FILE = os.path.join(BASE_PATH, "PO Data_formatted.xlsx")

def clean_and_format_data(input_file, output_file):
    # Read the CSV file
    print(f"Reading CSV file from: {input_file}")
    df = pd.read_csv(input_file)
    
    # Count initial rows
    initial_count = len(df)
    
    # Clean and format the data
    print("Cleaning and formatting data...")
    
    # 1. Format dates
    date_columns = ['BillDate', 'DueDate']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')
    
    # 2. Format currency amounts
    currency_columns = ['SubTotal', 'Total', 'Balance', 'Rate', 'ItemTotal']
    for col in currency_columns:
        df[col] = df[col].round(2)
        df[col] = df[col].apply(lambda x: f"{x:,.2f}")
    
    # 3. Clean up descriptions
    df['Description'] = df['Description'].replace('No Description', '')
    
    # 4. Remove duplicate rows
    df_cleaned = df.drop_duplicates()
    
    # 5. Sort the data by date and bill number
    df_cleaned = df_cleaned.sort_values(['BillDate', 'BillNumber'])
    
    # 6. Reorganize columns for better readability
    columns_order = [
        'BillNumber',
        'BillDate',
        'DueDate',
        'CustomerName',
        'VendorName',
        'SKU',
        'ItemName',
        'Description',
        'Quantity',
        'Unit',
        'Rate',
        'ItemTotal',
        'SubTotal',
        'Total',
        'Balance',
        'CurrencySymbol',
        'PaymentTerms',
        'Source',
        'CreatedTime'
    ]
    
    # Only include columns that exist in the dataframe
    columns_order = [col for col in columns_order if col in df_cleaned.columns]
    df_cleaned = df_cleaned[columns_order]
    
    # Generate summary statistics
    summary = {
        'Total Orders': df_cleaned['BillNumber'].nunique(),
        'Total Customers': df_cleaned['CustomerName'].nunique(),
        'Total Items': df_cleaned['SKU'].nunique(),
        'Date Range': f"{df_cleaned['BillDate'].min()} to {df_cleaned['BillDate'].max()}",
        'Total Records': len(df_cleaned),
        'Duplicates Removed': initial_count - len(df_cleaned)
    }
    
    # Create a pivot table for customer analysis
    customer_summary = pd.pivot_table(
        df_cleaned,
        index='CustomerName',
        values=['BillNumber', 'ItemTotal'],
        aggfunc={
            'BillNumber': 'nunique',
            'ItemTotal': lambda x: sum(float(str(i).replace(',', '')) for i in x)
        }
    ).round(2)
    
    customer_summary.columns = ['Total Orders', 'Total Amount']
    customer_summary = customer_summary.sort_values('Total Amount', ascending=False)
    
    # Save cleaned data to Excel file with formatting
    print(f"Saving formatted data to: {output_file}")
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Write main data
        df_cleaned.to_excel(writer, sheet_name='Orders', index=False)
        
        # Write summary
        pd.DataFrame([summary]).transpose().to_excel(writer, sheet_name='Summary')
        
        # Write customer summary
        customer_summary.to_excel(writer, sheet_name='Customer Analysis')
        
        # Get workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Orders']
        
        # Add formats
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'bg_color': '#D3D3D3',
            'border': 1
        })
        
        # Format the header row
        for col_num, value in enumerate(df_cleaned.columns.values):
            worksheet.write(0, col_num, value, header_format)
            
        # Auto-adjust columns' width
        for idx, col in enumerate(df_cleaned.columns):
            series = df_cleaned[col]
            max_len = max(
                series.astype(str).apply(len).max(),
                len(str(series.name))
            ) + 1
            worksheet.set_column(idx, idx, max_len)

    return summary

if __name__ == "__main__":
    try:
        # Ensure the path exists
        if not os.path.exists(INPUT_FILE):
            raise FileNotFoundError(f"Input file not found at: {INPUT_FILE}")
        
        summary = clean_and_format_data(INPUT_FILE, OUTPUT_FILE)
        
        print("\nData Processing Summary:")
        print("------------------------")
        for key, value in summary.items():
            print(f"{key}: {value}")
        print(f"\nFormatted data has been saved to: {OUTPUT_FILE}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [1]:
import pandas as pd
import numpy as np
import os

# Define the path
BASE_PATH = r"C:\Users\NyashaKampila\Desktop\Projects\Purchase-Order-Creation-Project"
INPUT_FILE = os.path.join(BASE_PATH, "PO Data.csv")
OUTPUT_FILE = os.path.join(BASE_PATH, "PO Data_formatted.xlsx")

def generate_vendor_id(vendor_name: str) -> str:
    """Generate a vendor ID based on vendor name"""
    # Remove special characters and spaces, take first 3 letters
    prefix = ''.join(c for c in vendor_name if c.isalnum())[:3].upper()
    # Add a unique number (using hash of full name to ensure consistency)
    hash_num = abs(hash(vendor_name)) % 1000
    return f"{prefix}{hash_num:03d}"

def generate_customer_id(customer_name: str) -> str:
    """Generate a customer ID based on customer name"""
    # Remove special characters and spaces, take first 3 letters
    prefix = ''.join(c for c in customer_name if c.isalnum())[:3].upper()
    # Add a unique number (using hash of full name to ensure consistency)
    hash_num = abs(hash(customer_name)) % 1000
    return f"{prefix}{hash_num:03d}"

def clean_and_format_data(input_file, output_file):
    # Read the CSV file
    print(f"Reading CSV file from: {input_file}")
    df = pd.read_csv(input_file)
    
    # Count initial rows
    initial_count = len(df)
    
    # Clean and format the data
    print("Cleaning and formatting data...")
    
    # Generate and add Vendor IDs
    print("Generating Vendor IDs...")
    vendor_ids = {name: generate_vendor_id(name) for name in df['VendorName'].unique()}
    df['VendorID'] = df['VendorName'].map(vendor_ids)
    
    # Generate and add Customer IDs
    print("Generating Customer IDs...")
    customer_ids = {name: generate_customer_id(name) for name in df['CustomerName'].unique()}
    df['CustomerID'] = df['CustomerName'].map(customer_ids)
    
    # Print the mappings for reference
    print("\nVendor ID Mappings:")
    for vendor, vid in vendor_ids.items():
        print(f"{vendor}: {vid}")
    
    print("\nCustomer ID Mappings:")
    for customer, cid in customer_ids.items():
        print(f"{customer}: {cid}")
    
    # 1. Format dates
    date_columns = ['BillDate', 'DueDate']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')
    
    # 2. Format currency amounts
    currency_columns = ['SubTotal', 'Total', 'Balance', 'Rate', 'ItemTotal']
    for col in currency_columns:
        df[col] = df[col].round(2)
        df[col] = df[col].apply(lambda x: f"{x:,.2f}")
    
    # 3. Clean up descriptions
    df['Description'] = df['Description'].replace('No Description', '')
    
    # 4. Remove duplicate rows
    df_cleaned = df.drop_duplicates()
    
    # 5. Sort the data by date and bill number
    df_cleaned = df_cleaned.sort_values(['BillDate', 'BillNumber'])
    
    # 6. Reorganize columns for better readability
    columns_order = [
        'BillNumber',
        'BillDate',
        'DueDate',
        'CustomerID',
        'CustomerName',
        'VendorID',
        'VendorName',
        'ItemName',
        'Description',
        'Quantity',
        'Unit',
        'Rate',
        'ItemTotal',
        'SubTotal',
        'Total',
        'Balance',
        'CurrencySymbol',
        'PaymentTerms',
        'Source',
        'CreatedTime'
    ]
    
    # Only include columns that exist in the dataframe
    columns_order = [col for col in columns_order if col in df_cleaned.columns]
    df_cleaned = df_cleaned[columns_order]
    
    # Verify no NaN values in ID columns
    if df_cleaned['VendorID'].isna().any() or df_cleaned['CustomerID'].isna().any():
        print("Warning: Found NaN values in ID columns!")
        print(f"NaN in VendorID: {df_cleaned['VendorID'].isna().sum()}")
        print(f"NaN in CustomerID: {df_cleaned['CustomerID'].isna().sum()}")
    
    # Generate summary statistics
    summary = {
        'Total Orders': df_cleaned['BillNumber'].nunique(),
        'Total Customers': df_cleaned['CustomerName'].nunique(),
        'Total Vendors': df_cleaned['VendorName'].nunique(),
        'Date Range': f"{df_cleaned['BillDate'].min()} to {df_cleaned['BillDate'].max()}",
        'Total Records': len(df_cleaned),
        'Duplicates Removed': initial_count - len(df_cleaned)
    }
    
    # Create a pivot table for customer analysis
    customer_summary = pd.pivot_table(
        df_cleaned,
        index=['CustomerID', 'CustomerName'],
        values=['BillNumber', 'ItemTotal'],
        aggfunc={
            'BillNumber': 'nunique',
            'ItemTotal': lambda x: sum(float(str(i).replace(',', '')) for i in x)
        }
    ).round(2)
    
    customer_summary.columns = ['Total Orders', 'Total Amount']
    customer_summary = customer_summary.sort_values('Total Amount', ascending=False)
    
    # Save cleaned data to Excel file with formatting
    print(f"Saving formatted data to: {output_file}")
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Write main data
        df_cleaned.to_excel(writer, sheet_name='Orders', index=False)
        
        # Write summary
        pd.DataFrame([summary]).transpose().to_excel(writer, sheet_name='Summary')
        
        # Write customer summary
        customer_summary.to_excel(writer, sheet_name='Customer Analysis')
        
        # Get workbook and worksheet objects
        workbook = writer.book
        worksheet = writer.sheets['Orders']
        
        # Add formats
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'top',
            'bg_color': '#D3D3D3',
            'border': 1
        })
        
        # Format the header row
        for col_num, value in enumerate(df_cleaned.columns.values):
            worksheet.write(0, col_num, value, header_format)
            
        # Auto-adjust columns' width
        for idx, col in enumerate(df_cleaned.columns):
            series = df_cleaned[col]
            max_len = max(
                series.astype(str).apply(len).max(),
                len(str(series.name))
            ) + 1
            worksheet.set_column(idx, idx, max_len)

    return summary

if __name__ == "__main__":
    try:
        # Ensure the path exists
        if not os.path.exists(INPUT_FILE):
            raise FileNotFoundError(f"Input file not found at: {INPUT_FILE}")
        
        summary = clean_and_format_data(INPUT_FILE, OUTPUT_FILE)
        
        print("\nData Processing Summary:")
        print("------------------------")
        for key, value in summary.items():
            print(f"{key}: {value}")
        print(f"\nFormatted data has been saved to: {OUTPUT_FILE}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Reading CSV file from: C:\Users\NyashaKampila\Desktop\Projects\Purchase-Order-Creation-Project\PO Data.csv
Cleaning and formatting data...
Generating Vendor IDs...
Generating Customer IDs...

Vendor ID Mappings:
Farmers Den Pack House Ltd: FAR559
Foodies Enterprise Zambia Ltd: FOO094
R & R Meats: RRM219
Uniturtle Farm: UNI510
Yalelo Zambia: YAL421
Abo Abbas Supermarket: ABO973
Nawa Nawa Farm: NAW566
Hrvst Emporium LTD: HRV168
Sea Pride: SEA345
Connicks Farms: CON471

Customer ID Mappings:
Cozy Zambia: COZ135
Freshbake Waddington: FRE211
Newrest: NEW117
Prime Joint Zambia: PRI627
Mocha n Chai: MOC317
Vida e Caffe Pinnacle: VID619
Controller Ciela Resort and Spa: CON442
Fox N Hound Restaurant: FOX601
Detohome Enterprise Limited: DET704
Vida e Cafe Cosmo: VID921
Vida E Cafe Zambezi: VID036
Urban Yard: URB100
Vida e Caffe East Park: VID474
Camren Catering: CAM248
Afridelivery: AFR953
Flame: FLA930
Copenhagen Yard Bar & Kitchen: COP982
Quick Service Restaurants LTD: QUI941
Millas Foods Zamb